In [ ]:
import numpy as np

# take in left and right wheel distance (traveled on the ground, not
# raw wheel rotations) and the distance between the wheels
# return the dx, dy, and heading change at t_1 for the t_0,
# body coordinate frame, e.g. given the body is in one location
# before the wheels move where will it be relative to that after they move?
# this is modelled as a circular motion, there is a spin center defined
# by the relative motion of the wheels, it is off at infinity when traveling
# in a straight line
def wheel_distance_to_state_change(
    left_distance: float,
    right_distance: float,
    wheel_base: float,
) -> (float, float, float):
    theta = (right_distance - left_distance) / wheel_base
    center_distance = (right_distance + left_distance) / 2.0
    
    dx = center_distance * np.sinc(theta / np.pi)
    dy = dx * np.tan(theta / 2.0)
    
    return (dx, dy, theta, center_distance)

t0 = 0.0
t1 = 2.0

wheel_base = 0.1

dts = [0.00011, 0.001, 0.01, 0.017, 0.045, 0.09]

import matplotlib
import matplotlib.pyplot as plt

fig0 = plt.figure(figsize=(12, 12))
ax0 = fig0.add_subplot(1, 1, 1)

fig1 = plt.figure(figsize=(14, 5))
ax1 = fig1.add_subplot(1, 1, 1)
plt.ylabel('left/right distance')

fig2 = plt.figure(figsize=(14, 5))
ax2 = fig2.add_subplot(1, 1, 1)
plt.ylabel('x')

fig3 = plt.figure(figsize=(14, 5))
ax3 = fig3.add_subplot(1, 1, 1)
plt.ylabel('y')

fig4 = plt.figure(figsize=(14, 5))
ax4 = fig4.add_subplot(1, 1, 1)
plt.ylabel('theta/heading')

for dt in dts:
    ts = np.arange(t0, t1, dt)
    num = ts.shape[0]
    print(f"{dt} {num}")
    ramp = np.arange(0, num) / num
    dist_left = 14.0 * ramp
    dist_left += 0.2 * np.sin(ts * 2.569) * ramp
    dist_left -= dist_left[0]
    # dist_right = abs(np.sin(ts * 1.9))
    dist_right = 14.0 * ramp
    dist_right -= dist_right[0]

    # TODO(lucasw) vectorize

    # initial state
    x = 0.0
    y = 0.0
    theta = 0.0
    xs = []
    ys = []
    cs = []
    thetas = []
    last_l = dist_left[0]
    last_r = dist_right[0]
    dist_traveled = 0.0
    for ind, (t, l, r) in enumerate(zip(ts, dist_left, dist_right)):
        dl = l - last_l
        dr = r - last_r
        last_l = l
        last_r = r
        
        dx_in_body, dy_in_body, dtheta, c_dist = wheel_distance_to_state_change(dl, dr, wheel_base)
        dist_traveled += c_dist
        
        x += dx_in_body * np.cos(theta) - dy_in_body * np.sin(theta)
        y += dx_in_body * np.sin(theta) + dy_in_body * np.cos(theta)
        theta += dtheta

        cs.append(dist_traveled)
        xs.append(x)
        ys.append(y)
        thetas.append(theta)

    print(f"dt: {dt}s, distance traveled: {dist_traveled:.6}")
    ax0.plot(xs, ys, '-.')
    ax1.plot(ts, dist_left, '.')
    ax1.plot(ts, dist_right, '.')
    ax2.plot(ts, xs, '.')
    ax3.plot(ts, ys, '.')
    ax4.plot(ts, thetas, '.')
    
ax0.plot(0.0, 0.0, 'o')
ax0.plot(xs[-1], ys[-1], 'o')
ax0.axis('equal')

In [ ]:
# TODO(lucasw) try fitting clothoid segments to the above curve

In [ ]:
# get the distance traveled along the sequence of points
def get_distance(xs, ys):
    dx = xs[1:] - xs[:-1]
    dx2 = np.multiply(dx, dx)
    dy = ys[1:] - ys[:-1]
    dy2 = np.multiply(dy, dy)
    
    d2 = np.sqrt(dx2 + dy2)
    distance = np.sum(d2)
    return distance
    
def plot_turn(center_radius, theta, wheel_base=0.5, num=20):
    heading = theta + np.pi / 2.0
    
    angles = []
    for i in range(num + 1):
        fr = i / num
        angle = theta * fr
        angles.append(angle)

    center_x = center_radius * np.cos(angles)
    center_y = center_radius * np.sin(angles)

    left_r = center_radius - wheel_base / 2.0
    right_r = center_radius + wheel_base / 2.0

    left_x = left_r * np.cos(angles)
    left_y = left_r * np.sin(angles)
    
    right_x = right_r * np.cos(angles)
    right_y = right_r * np.sin(angles)

    ax0.plot(left_x, left_y)
    ax0.plot(right_x, right_y)
    
    ax0.plot([0, right_x[0]], [0, right_y[0]])
    ax0.plot([0, right_x[-1]], [0, right_y[-1]])

    ax0.plot(center_x, center_y)
    
    hdx = np.cos(heading)
    hdy = np.sin(heading)
    sc = 0.03
    ax0.arrow(left_x[-1], left_y[-1], sc * hdx, sc * hdy, head_width=0.025)
    ax0.arrow(center_x[-1], center_y[-1], sc * hdx, sc * hdy, head_width=0.025)
    ax0.arrow(right_x[-1], right_y[-1], sc * hdx, sc * hdy, head_width=0.025)

    ax0.plot([left_x[0], right_x[0]], [left_y[0], right_y[0]], linewidth=4)
    ax0.text(center_x[0], center_y[0] - 0.05, "wheel base", ha="center")

    ax0.text(center_x[num // 2], center_y[num // 2], f"center\nr={center_radius:.3}")
    ax0.text(right_x[num // 2], right_y[num // 2], f"right\nr={right_r:.3}")
    ax0.text(left_x[num // 2], left_y[num // 2], f"left\nr={left_r:.3}")
    ax0.text(0.15, 0.05, f"θ = {theta/np.pi:.3}π")

    ax0.axis('equal')

    left_distance = get_distance(left_x, left_y)
    left_dist2 = left_r * theta
    print(f"left_distance integrated: {left_distance:.3}, left_r * θ = {left_r:.3} * {theta:.3} = {left_dist2:.3}")
    # ax0.plot([left_x[0], left_x[0]], [left_y[0], left_y[0] + left_dist2])
    
    right_distance = get_distance(right_x, right_y)
    right_dist2 = right_r * theta
    print(f"right_distance integrated: {right_distance:.3}, right_r * θ = {right_r:.3} * {theta:.3} = {right_dist2:.3}")
    
    ax0.plot([left_x[-1], left_x[-1] + wheel_base], [left_y[-1], left_y[-1]], '-.')
    right2_x = wheel_base * np.cos(angles)
    right2_y = wheel_base * np.sin(angles)
    lr_distance = get_distance(right2_x, right2_y)
    print(f"lr distance {lr_distance:0.3}, right - left ({right_dist2 - left_dist2:.3})")
    ax0.plot(left_x[-1] + right2_x, left_y[-1] + right2_y, '-.')
    
    theta_mid = left_r / right_r * theta
    ax0.plot([0, right_r * np.cos(theta_mid)], [0, right_r * np.sin(theta_mid)], '-.')

center_radius = 1.2
theta = np.pi / 5.0

fig0 = plt.figure(figsize=(8, 8))
ax0 = fig0.add_subplot(1, 1, 1)

plot_turn(center_radius, theta)